In [7]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import gc
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge

In [8]:
LGB1 = pd.read_csv("[190130]LGB_Quantile_not_deep.csv") #Quantile SAMPLING
LGB1.columns = ['key','LGB1']

LGB4 = pd.read_csv("LGB_Testfile_LB5150_LV.csv") #LAST VALIDATION SAMPLING
LGB4.columns = ['key','LGB4']

XGB1 = pd.read_csv("01.30.XGB_Bestscore-2.csv") #5fold NOT DEEP NOT SAMPLING
XGB1.columns = ['key','XGB1']

XGB4 = pd.read_csv("01.29.Xgb_3fold.csv") #3fold NOT DEEP NOT SAMPLING
XGB4.columns = ['key','XGB4']

In [9]:
SUB = pd.merge(LGB1,LGB4,on='key')
SUB = pd.merge(SUB,XGB1,on='key')
SUB = pd.merge(SUB,XGB4,on='key')

In [10]:
train = pd.read_csv("train.csv")[['key','transaction_real_price','city']]
test = pd.read_csv("test.csv")[['key','transaction_real_price','city']]
test.loc[test['key']==1503614,'city'] = 0

del test['transaction_real_price']

busan_train = train[train['city']==0].reset_index(drop=True)
busan_test = test[test['city']==0].reset_index(drop=True)

seoul_train = train[train['city']==1].reset_index(drop=True)
seoul_test = test[test['city']==1].reset_index(drop=True)

SUB_BUSAN = pd.merge(busan_test,SUB,on='key',how='left')
SUB_SEOUL = pd.merge(seoul_test,SUB,on='key',how='left')
del SUB_BUSAN['city']
del SUB_SEOUL['city']

In [11]:
SUB_BUSAN['LGB1_XGB1'] = 0.8*SUB_BUSAN['LGB1'] + 0.2*SUB_BUSAN['XGB1'] 
SUB_SEOUL['LGB1_XGB1'] = 0.7*SUB_SEOUL['LGB1'] + 0.3*SUB_SEOUL['XGB1'] 

SUB_BUSAN['LGB1_XGB4'] = 0.9*SUB_BUSAN['LGB1'] + 0.1*SUB_BUSAN['XGB4'] 
SUB_SEOUL['LGB1_XGB4'] = 0.7*SUB_SEOUL['LGB1'] + 0.3*SUB_SEOUL['XGB4'] 

SUB_BUSAN['transaction_real_price'] = 2.1*SUB_BUSAN['LGB1_XGB1']  -0.4*SUB_BUSAN['LGB4']  - 0.7*SUB_BUSAN['LGB1_XGB4'] 
SUB_SEOUL['transaction_real_price'] = 0.1*SUB_SEOUL['LGB1_XGB1']  -0.3*SUB_SEOUL['LGB4']  + 1.2*SUB_SEOUL['LGB1_XGB4'] 

In [12]:
SUB = pd.concat([SUB_BUSAN,SUB_SEOUL])
SUB = SUB[['key','transaction_real_price']]
SUB.to_csv("LAST_SOLUTION.csv",index=False)